In [4]:
import os
import warnings
warnings.filterwarnings("ignore")
import torch
from torch import Tensor
import torch.nn as nn
import pandas as pd
import numpy as np
import random

In [2]:
base_data_dir = '../../data'
word2vec_dir = 'word2vec/ptb'
ptb_train_fname = 'ptb.train.txt'

In [5]:
with open(os.path.join(base_data_dir, word2vec_dir, ptb_train_fname)) as f:
    raw_text = f.read()

In [9]:
sentences = [line.split() for line in raw_text.split('\n')]